In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Import libraries

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
import numpy as np
import pandas as pd

Read all the files and create three dataset, each one about a component of the velocity (respectively Ux,Uy and Uz) for every points and for every seconds: on the rows, there are the 11160 points, while, on the columns, the velocity component for the 10010 seconds (taken at interval of 10s).

In [ ]:
# insert your path
path='./drive/MyDrive/Colab_Notebooks/Datamining_Project/'

In [ ]:
li_1 = []
li_2 = []
li_3 = []
for i in range(1001):
    df = pd.read_csv(path+'data/dataFile0.'+str(i)+'.csv', index_col=None, header=0)
    df.set_index(keys=['Points:0','Points:1','Points:2'], inplace=True)
    li_1.append(df['U:0'].to_frame())
    li_2.append(df['U:1'].to_frame())
    li_3.append(df['U:2'].to_frame())
    if i==0:
        li_1[0].rename(columns={"U:0":' '.join(['U:0',str(0)])}, inplace=True)
        li_2[0].rename(columns={"U:1":' '.join(['U:1',str(0)])}, inplace=True)
        li_3[0].rename(columns={"U:2":' '.join(['U:2',str(0)])}, inplace=True)
        tot1=li_1[0]
        tot2=li_2[0]
        tot3=li_3[0]
    else:
        li_1[i].rename(columns={"U:0":' '.join(['U:0',str(i)])}, inplace=True)
        li_2[i].rename(columns={"U:1":' '.join(['U:1',str(i)])}, inplace=True)
        li_3[i].rename(columns={"U:2":' '.join(['U:2',str(i)])}, inplace=True)
        tot1 = tot1.join(li_1[i])
        tot2 = tot2.join(li_2[i])
        tot3 = tot3.join(li_3[i])

In [ ]:
 # only to save data
 # tot1.to_csv(path+'tot_velocity_x.csv", index=[['Points:0','Points:1','Points:2']])
 # tot2.to_csv(path+'tot_velocity_y.csv", index=[['Points:0','Points:1','Points:2']])
 # tot3.to_csv(path+'tot_velocity_z.csv", index=[['Points:0','Points:1','Points:2']])

Define the function to prepare data for the LSTM model.

In [ ]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
    X, Y = list(), list()
    for i in range(len(sequences)+1):
        # find the end of this pattern
        end_ix = i + n_steps
        
        # check if we are beyond the dataset
        if end_ix > len(sequences)-1:
            break
		
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :]
        X.append(seq_x)
        Y.append(seq_y)
        
    return np.array(X), np.array(Y) # the outputs of the function are, respectively, the input data and the target data of the model

--------------------------------------------------------------------------------
**Component x**

In [ ]:
# only to read data
tot1=pd.read_csv(path+'tot_velocity_x.csv',index_col=['Points:0','Points:1','Points:2'])
tot_df1 = tot1.T
tot1 = tot_df1.to_numpy()

Check if there are any missing values.

In [ ]:
tot_df1.T[tot_df1.isna().any()]

,,,U:0 0,U:0 1,U:0 2,U:0 3,U:0 4,U:0 5,U:0 6,U:0 7,U:0 8,U:0 9,U:0 10,U:0 11,U:0 12,U:0 13,U:0 14,U:0 15,U:0 16,U:0 17,U:0 18,U:0 19,U:0 20,U:0 21,U:0 22,U:0 23,U:0 24,U:0 25,U:0 26,U:0 27,U:0 28,U:0 29,U:0 30,U:0 31,U:0 32,U:0 33,U:0 34,U:0 35,U:0 36,U:0 37,U:0 38,U:0 39,...,U:0 961,U:0 962,U:0 963,U:0 964,U:0 965,U:0 966,U:0 967,U:0 968,U:0 969,U:0 970,U:0 971,U:0 972,U:0 973,U:0 974,U:0 975,U:0 976,U:0 977,U:0 978,U:0 979,U:0 980,U:0 981,U:0 982,U:0 983,U:0 984,U:0 985,U:0 986,U:0 987,U:0 988,U:0 989,U:0 990,U:0 991,U:0 992,U:0 993,U:0 994,U:0 995,U:0 996,U:0 997,U:0 998,U:0 999,U:0 1000
Points:0,Points:1,Points:2,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


Summary statistics of the dataset

In [ ]:
desc=tot_df1.describe()

In [ ]:
desc

Points:0      -24.750                -25.000  ...   64.389   64.694       65.000
Points:1     -9.3333  -10.0000      -9.3333   ...  10.0000  10.0000      10.0000
Points:2            0        0             0  ...        0        0            0
count     1001.000000   1001.0  1.001000e+03  ...   1001.0   1001.0  1001.000000
mean         0.049947      0.0  5.000000e-02  ...      0.0      0.0     0.022429
std          0.001580      0.0  7.081210e-16  ...      0.0      0.0     0.002405
min          0.000000      0.0  5.000000e-02  ...      0.0      0.0     0.000000
25%          0.049998      0.0  5.000000e-02  ...      0.0      0.0     0.021174
50%          0.049998      0.0  5.000000e-02  ...      0.0      0.0     0.021888
75%          0.049998      0.0  5.000000e-02  ...      0.0      0.0     0.023045
max          0.049999      0.0  5.000000e-02  ...      0.0      0.0     0.031230

[8 rows x 11160 columns]

In [ ]:
max(desc.T['max'])

0.089588

In [ ]:
min(desc.T['min'])

-0.050659

Splitting data into training set - test set

In [ ]:
# Training set - Test set
# 80% - 20%, choosed chronologically

tot_train = tot1[:800,:]
tot_test = tot1[800:,:]

Build the model

In [ ]:
# choose a number of time steps (trade off computational between cost and quality of the model)
n_steps = 75
# convert into input/output
X_train, Y_train = split_sequences(tot_train, n_steps)
n_features = X_train.shape[2]
# define model
model = Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model.add(LSTM(100, activation='relu'))
model.add(Dense(n_features))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X_train, Y_train, epochs=400, verbose=0)
# convert into input/output
X_test, Y_test = split_sequences(tot_test, n_steps)

Finally compute the prediction

In [ ]:
Y_pred = model.predict(X_test, verbose=0)

Save data

In [ ]:
# only to save data
# pd.DataFrame(Y_pred).to_csv(path+'Y_vel_lstm.csv', index=False)

In [ ]:
# only to read data
# Y_pred = pd.read_csv(path+'Y_vel_lstm.csv')

Update the 126 csv files with the predictions.

In [ ]:
Y_pred = Y_pred.to_numpy()
# update dataset that contains predicted pression with predicted velocity
for i in range(len(Y_pred)):
  j=n_steps+i
  data_pred = pd.read_csv(path+'dataLSTM/dataPred_'+str(1000-200+j)+'.csv')
  data_pred = data_pred[['Predicted p','Points:0','Points:1','Points:2']]
  data_pred.insert(1,'Predicted U:0',Y_pred[i])
  data_pred.to_csv(path+"dataLSTM/dataPred_"+str(1000-200+j)+".csv", index=False)

--------------------------------------------------------------------------------
**Component y**

In [ ]:
# only to read data
tot_df2 = pd.read_csv(path+'tot_velocity_y.csv', index_col=['Points:0','Points:1','Points:2'])
tot_df2 = tot_df2.T
tot2 = tot_df2.to_numpy()

Check if there are any missing values.

In [ ]:
tot_df2.T[tot_df2.isna().any()]

,,,U:1 0,U:1 1,U:1 2,U:1 3,U:1 4,U:1 5,U:1 6,U:1 7,U:1 8,U:1 9,U:1 10,U:1 11,U:1 12,U:1 13,U:1 14,U:1 15,U:1 16,U:1 17,U:1 18,U:1 19,U:1 20,U:1 21,U:1 22,U:1 23,U:1 24,U:1 25,U:1 26,U:1 27,U:1 28,U:1 29,U:1 30,U:1 31,U:1 32,U:1 33,U:1 34,U:1 35,U:1 36,U:1 37,U:1 38,U:1 39,...,U:1 961,U:1 962,U:1 963,U:1 964,U:1 965,U:1 966,U:1 967,U:1 968,U:1 969,U:1 970,U:1 971,U:1 972,U:1 973,U:1 974,U:1 975,U:1 976,U:1 977,U:1 978,U:1 979,U:1 980,U:1 981,U:1 982,U:1 983,U:1 984,U:1 985,U:1 986,U:1 987,U:1 988,U:1 989,U:1 990,U:1 991,U:1 992,U:1 993,U:1 994,U:1 995,U:1 996,U:1 997,U:1 998,U:1 999,U:1 1000
Points:0,Points:1,Points:2,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


Summary statistics of the dataset

In [ ]:
desc=tot_df2.describe()

In [ ]:
desc

Points:0      -24.750           -25.000  ...   64.389   64.694       65.000
Points:1     -9.3333  -10.0000 -9.3333   ...  10.0000  10.0000      10.0000
Points:2            0        0        0  ...        0        0            0
count     1001.000000   1001.0   1001.0  ...   1001.0   1001.0  1001.000000
mean         0.000011      0.0      0.0  ...      0.0      0.0     0.000010
std          0.000018      0.0      0.0  ...      0.0      0.0     0.000331
min          0.000000      0.0      0.0  ...      0.0      0.0    -0.001033
25%          0.000009      0.0      0.0  ...      0.0      0.0    -0.000117
50%          0.000010      0.0      0.0  ...      0.0      0.0    -0.000074
75%          0.000010      0.0      0.0  ...      0.0      0.0     0.000009
max          0.000505      0.0      0.0  ...      0.0      0.0     0.002941

[8 rows x 11160 columns]

In [ ]:
max(desc.T['max'])

0.0478149999999999

In [ ]:
min(desc.T['min'])

-0.061338

Splitting data into training set - test set

In [ ]:
# Training set - Test set
# 80% - 20%, choosed chronologically

tot_train = tot2[:800,:]
tot_test = tot2[800:,:]

Build the model

In [ ]:
# choose a number of time steps (trade off computational between cost and quality of the model)
n_steps = 75
# convert into input/output
X_train, Y_train = split_sequences(tot_train, n_steps)
n_features = X_train.shape[2]
# define model
model = Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model.add(LSTM(100, activation='relu'))
model.add(Dense(n_features))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X_train, Y_train, epochs=400, verbose=0)
# convert into input/output
X_test, Y_test = split_sequences(tot_test, n_steps)

5.448012529996049e-07


Finally compute the prediction

In [ ]:
Y_pred = model.predict(X_test, verbose=0)

Save data

In [ ]:
# only to save data
# pd.DataFrame(Y_pred).to_csv(path+"Y_vel2_lstm.csv", index=False)

In [ ]:
# only to read data
# Y_pred = pd.read_csv(path+"Y_vel2_lstm.csv")

Update the 126 csv files with the predictions.

In [ ]:
Y_pred = Y_pred.to_numpy()
# update dataset that contains predicted pression with predicted velocity
for i in range(len(Y_pred)):
  j=n_steps+i
  data_pred = pd.read_csv(path+'dataLSTM/dataPred_'+str(1000-200+j)+'.csv')
  data_pred.insert(2,'Predicted U:1',Y_pred[i])
  data_pred.to_csv(path+"dataLSTM/dataPred_"+str(1000-200+j)+".csv", index=False)

--------------------------------------------------------------------------------
**Component z**

In [ ]:
# only to read data
tot_df3 = pd.read_csv(path+'tot_velocity_z.csv', index_col=['Points:0','Points:1','Points:2'])
tot_df3 = tot_df3.T
tot3 = tot_df3.to_numpy()

Check if there are any missing values.

In [ ]:
tot_df3.T[tot_df3.isna().any()]

,,,U:2 0,U:2 1,U:2 2,U:2 3,U:2 4,U:2 5,U:2 6,U:2 7,U:2 8,U:2 9,U:2 10,U:2 11,U:2 12,U:2 13,U:2 14,U:2 15,U:2 16,U:2 17,U:2 18,U:2 19,U:2 20,U:2 21,U:2 22,U:2 23,U:2 24,U:2 25,U:2 26,U:2 27,U:2 28,U:2 29,U:2 30,U:2 31,U:2 32,U:2 33,U:2 34,U:2 35,U:2 36,U:2 37,U:2 38,U:2 39,...,U:2 961,U:2 962,U:2 963,U:2 964,U:2 965,U:2 966,U:2 967,U:2 968,U:2 969,U:2 970,U:2 971,U:2 972,U:2 973,U:2 974,U:2 975,U:2 976,U:2 977,U:2 978,U:2 979,U:2 980,U:2 981,U:2 982,U:2 983,U:2 984,U:2 985,U:2 986,U:2 987,U:2 988,U:2 989,U:2 990,U:2 991,U:2 992,U:2 993,U:2 994,U:2 995,U:2 996,U:2 997,U:2 998,U:2 999,U:2 1000
Points:0,Points:1,Points:2,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


Summary statistics of the dataset

In [ ]:
desc=tot_df3.describe()

In [ ]:
desc

Points:0  -24.750           -25.000  ...   64.389   64.694   65.000
Points:1 -9.3333  -10.0000 -9.3333   ...  10.0000  10.0000  10.0000
Points:2        0        0        0  ...        0        0        0
count      1001.0   1001.0   1001.0  ...   1001.0   1001.0   1001.0
mean          0.0      0.0      0.0  ...      0.0      0.0      0.0
std           0.0      0.0      0.0  ...      0.0      0.0      0.0
min           0.0      0.0      0.0  ...      0.0      0.0      0.0
25%           0.0      0.0      0.0  ...      0.0      0.0      0.0
50%           0.0      0.0      0.0  ...      0.0      0.0      0.0
75%           0.0      0.0      0.0  ...      0.0      0.0      0.0
max           0.0      0.0      0.0  ...      0.0      0.0      0.0

[8 rows x 11160 columns]

In [ ]:
max(desc.T['max'])

2.9409e-17

In [ ]:
min(desc.T['min'])

-2.9151e-17

Splitting into training set - test set

In [ ]:
# Training set - Test set
# 80% - 20%, choosed chronologically

tot_train = tot3[:800,:]
tot_test = tot3[800:,:]

Build the model

In [ ]:
# choose a number of time steps (trade off computational between cost and quality of the model)
n_steps = 75
# convert into input/output
X_train, Y_train = split_sequences(tot_train, n_steps)
n_features = X_train.shape[2]
# define model
model = Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model.add(LSTM(100, activation='relu'))
model.add(Dense(n_features))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X_train, Y_train, epochs=400, verbose=0)
# convert into input/output
X_test, Y_test = split_sequences(tot_test, n_steps)

0.0


Finally compute the prediction

In [ ]:
Y_pred = model.predict(X_test, verbose=0)

Save data

In [ ]:
# only to save data
# pd.DataFrame(Y_pred).to_csv(path+"Y_vel3_lstm.csv", index=False)

In [ ]:
# only to read data
# Y_pred = pd.read_csv(path+"Y_vel3_lstm.csv")

Update the 126 csv files with the predictions.

In [ ]:
Y_pred = Y_pred.to_numpy()
# update dataset that contains predicted pression with predicted velocity
for i in range(len(Y_pred)):
  j=i+n_steps
  data_pred = pd.read_csv(path+'dataLSTM/dataPred_'+str(1000-200+j)+'.csv')
  data_pred.insert(3,'Predicted U:2',Y_pred[i])
  data_pred.to_csv(path+"dataLSTM/dataPred_"+str(1000-200+j)+".csv", index=False)

ValueError: ignored